In [4]:
from notebook_inits import *
from pathlib import Path
from main import *
import pandas as pd

In [5]:
root == str(Path.cwd())

True

In [6]:
all_data = fetch_all_data()
cb_data = all_data['cb']

fetching exchange data.. https://api.exchange.coinbase.com/p
cb fetch complete
fetching exchange data.. https://api.gemini.com/v1/book/BTCU
Finished processing and saving data for: cb
gem fetch complete
Finished processing and saving data for: gem


In [7]:
print(cb_data.keys())
print(cb_data['sequence'], cb_data['auction_mode'], cb_data['auction'], cb_data['time'])
('asks', len(cb_data['asks'])), ('bids', len(cb_data['bids']))

dict_keys(['bids', 'asks', 'sequence', 'auction_mode', 'auction', 'time'])
114835409651 False None 2025-11-04T10:10:37.024259059Z


(('asks', 23548), ('bids', 19355))

### Approach1, simple iterative: calculates execution price for every call

In [8]:
def calculate_price(orders:list, target_qty:float=10): # specific to coinbase
    # assuming orders are already sorted ascending by price
    remaining_qty: float = target_qty
    order_value: float = 0.0
    for order in orders:
        if remaining_qty <= 0:
            break
        order_price, order_qty = float(order[0]), float(order[1])
        available_qty  = min(order_qty, remaining_qty)
        order_value   += available_qty*order_price
        remaining_qty -= available_qty
        # print([order_price, order_qty, available_qty, remaining_qty])
    if remaining_qty > 0:
        print(f'\tPartial order:\n\tfilled={target_qty-remaining_qty};\n\t{remaining_qty=}')
       
    return order_value

In [9]:
%%timeit -r1 -n1
target_qt=100000
print(f'To buy  {target_qt} BTC= $', calculate_price(cb_data['asks'], target_qty=target_qt))
print('*'*12)
print(f'To sell {target_qt} BTC= $', calculate_price(cb_data['bids'], target_qty=target_qt))
print('='*25)

	Partial order:
	filled=4837.545136991481;
	remaining_qty=95162.45486300852
To buy  100000 BTC= $ 929741050.1100509
************
To sell 100000 BTC= $ 267083389.04967645
7.89 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Approach2, pandas.DataFrame: keeps the execution price pre-calculated & finds the price in between (minimal calculations)

In [10]:
# ASKS <-- lowest price: seller willing to accept; use it for buying
cols= ['price', 'qty', 'ods']
cb_asks = pd.DataFrame(cb_data['asks'], columns=cols)
del cb_asks['ods']
cb_asks[cols[:2]] = cb_asks[cols[:2]].astype(float)
cb_asks

,price,qty
0,1.037137e+05,1.000000e+00
1,1.037160e+05,1.070000e-02
2,1.037180e+05,1.070000e-02
3,1.037191e+05,2.312200e-04
4,1.037217e+05,9.637000e-04
...,...,...
23543,3.690808e+07,7.000000e-08
23544,9.999000e+07,1.632500e-04
23545,1.100239e+08,4.000000e-08
23546,1.238164e+08,1.100000e-07


In [11]:
# BIDS <-- highest price: buyers willing to pay; use it for selling
cols= ['price', 'qty', 'ods']
cb_bids = pd.DataFrame(cb_data['bids'], columns=cols)
del cb_bids['ods']
cb_bids[cols[:2]] = cb_bids[cols[:2]].astype(float)
cb_bids

,price,qty
0,103713.71,2.046016e-02
1,103712.87,9.642000e-03
2,103710.90,7.711514e-02
3,103710.13,9.636700e-04
4,103707.22,2.819777e-02
...,...,...
19350,0.05,1.519500e+04
19351,0.04,1.000000e+02
19352,0.03,3.442314e+04
19353,0.02,5.400000e+03


In [13]:
gem_data = all_data['gem']
gem_data.keys()

dict_keys(['bids', 'asks'])

In [14]:
gem_bids, gem_asks = pd.DataFrame(gem_data['bids']), pd.DataFrame(gem_data['asks'])
del gem_bids['timestamp']
del gem_asks['timestamp']

In [15]:
gem_asks[:4]

,price,amount
0,103717.55,0.02531014
1,103723.9,0.01
2,103723.91,0.10124156
3,103729.78,0.017338


In [16]:
gem_bids[:4]

,price,amount
0,103703.69,0.00192
1,103703.57,0.00192
2,103703.38,0.00481
3,103698.51,0.01


In [17]:
gem_cols = ['price', 'qty']
gem_bids.rename(columns={'amount': 'qty'}, inplace=True)
gem_bids[gem_cols] = gem_bids[gem_cols].astype(float)

gem_asks.rename(columns={'amount': 'qty'}, inplace=True)
gem_asks[gem_cols] = gem_asks[gem_cols].astype(float)

In [18]:
gem_asks[:4]

,price,qty
0,103717.55,0.025310
1,103723.90,0.010000
2,103723.91,0.101242
3,103729.78,0.017338


In [19]:
gem_bids[:4]

,price,qty
0,103703.69,0.00192
1,103703.57,0.00192
2,103703.38,0.00481
3,103698.51,0.01000


In [20]:
merged_bids = pd.concat((cb_bids, gem_bids)).dropna().groupby('price').sum().reset_index()
merged_bids = merged_bids.sort_values(by='price', ascending=False, ignore_index=True)

In [21]:
merged_asks = pd.concat((cb_asks, gem_asks)).dropna().groupby('price').sum().reset_index()
merged_asks = merged_asks.sort_values(by='price', ascending=True, ignore_index=True)

In [22]:
merged_bids['cum_sum_qty'] = merged_bids['qty'].cumsum()
merged_bids['cum_sum_execution_price'] = (merged_bids['price']*merged_bids['qty']).cumsum()
# ---
merged_asks['cum_sum_qty'] = merged_asks['qty'].cumsum()
merged_asks['cum_sum_execution_price'] = (merged_asks['price']*merged_asks['qty']).cumsum()

In [23]:
merged_bids, len(merged_bids)

(           price           qty   cum_sum_qty  cum_sum_execution_price
 0      103713.71  2.046016e-02  2.046016e-02             2.121999e+03
 1      103712.87  9.642000e-03  3.010216e-02             3.121999e+03
 2      103710.90  7.711514e-02  1.072173e-01             1.111968e+04
 3      103710.13  9.636700e-04  1.081810e-01             1.121962e+04
 4      103707.22  2.819777e-02  1.363787e-01             1.414393e+04
 ...          ...           ...           ...                      ...
 19393       0.05  1.519500e+04  4.185810e+04             2.686316e+08
 19394       0.04  1.000000e+02  4.195810e+04             2.686316e+08
 19395       0.03  3.442314e+04  7.638123e+04             2.686326e+08
 19396       0.02  5.400000e+03  8.178123e+04             2.686327e+08
 19397       0.01  3.132680e+06  3.214461e+06             2.686641e+08
 
 [19398 rows x 4 columns],
 19398)

In [24]:
def calculate_execution_price_df(orders: pd.DataFrame, target_qt: float=10) -> float:
    if orders.empty:
        return 0
    insert_position = orders['cum_sum_qty'].searchsorted(target_qt)
    print(f">> {insert_position=}")
    print(orders[insert_position-1: insert_position+1])
    if insert_position == 0: # means lesser quantity than order book
        print(orders.iloc[0], 'target=', target_qt)
        return target_qt * orders.iloc[0]['price']

    if insert_position >= len(orders) - 1: # more than order book quantity, leads to partial fill the target_qty
        print(f"Partial fill --{target_qt=}; --remaining_qty={target_qt - orders.iloc[-1]['cum_sum_qty']}")
        return orders.iloc[-1]['cum_sum_execution_price']
    
    # somewhere in between the orders
    # print(orders[insert_position-1: insert_position+1])
    for idx, row in orders[insert_position-1: insert_position+1].iterrows():
        # find 2 exact rows where target_qty would fit
        print(f'processing row_idx={idx}')
        print('\n\t---', row['cum_sum_qty'])
        if row['cum_sum_qty'] == target_qt:
            print('------- found exact quantity')
            return row['cum_sum_execution_price']
        if row['cum_sum_qty'] < target_qt < orders.iloc[idx+1]['cum_sum_qty']:
            print(f'Found exact position: {(idx, idx+1)}')
            return row['cum_sum_execution_price'] + ((orders.iloc[idx+1]['cum_sum_qty'] - target_qt) * orders.iloc[idx+1]['price'])
    

In [25]:
%%timeit -n1 -r1
print(calculate_execution_price_df(merged_asks, 1))
print(calculate_execution_price_df(merged_bids, 1))

>> insert_position=np.int64(0)
Empty DataFrame
Columns: [price, qty, cum_sum_qty, cum_sum_execution_price]
Index: []
price                      103713.72
qty                             1.00
cum_sum_qty                     1.00
cum_sum_execution_price    103713.72
Name: 0, dtype: float64 target= 1
103713.72
>> insert_position=np.int64(30)
        price       qty  cum_sum_qty  cum_sum_execution_price
29  103694.40  0.050000     0.985815            102231.641755
30  103694.25  0.101242     1.087057            112729.809388
processing row_idx=29

	--- 0.98581524
Found exact position: (29, 30)
111258.93133834962
3.34 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [22]:
# calculate_execution_price_df(merged_bids, 0.1)

In [26]:
merged_asks.to_csv('merged_asks.csv')

In [27]:
merged_bids.to_csv('merged_bids.csv')